In [ ]:
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import numpy as np 
zip_path = "march-machine-learning-mania-2025.zip"
inner_file = "MRegularSeasonDetailedResults.csv"
conference_file = "MTeamConferences.csv"
mmassey_rankings = 'MMasseyOrdinals.csv'

with zipfile.ZipFile(zip_path) as z:
    with z.open(inner_file) as f1, z.open(conference_file) as f2, z.open(mmassey_rankings) as f3:
        df_teams = pd.read_csv(f1)
        df_conferences = pd.read_csv(f2)
        df_massey = pd.read_csv(f3)

In [ ]:
TARGET_CONFERENCES = ['acc', 'big_twelve', 'sec', 'big_ten', 'pac_twelve', 'big_east']
teams_in_target_confs = df_conferences[
    df_conferences['ConfAbbrev'].isin(TARGET_CONFERENCES)
]['TeamID'].unique()

In [ ]:
def calculate_point_spread(df_teams):
    """
    Calculate home team point spread correctly
    WLoc refers to the WINNING team's location:
    - 'H': Winner was home team
    - 'A': Winner was away team  
    - 'N': Neutral site
    """
    df = df_teams.copy()
    
    # Determine home and away teams based on WLoc
    # If WLoc == 'H': WTeamID is home, LTeamID is away
    # If WLoc == 'A': LTeamID is home, WTeamID is away
    # If WLoc == 'N': Neutral site (we'll use WTeamID as "team1", LTeamID as "team2")
    
    df['HomeTeamID'] = np.where(df['WLoc'] == 'H', df['WTeamID'],
                        np.where(df['WLoc'] == 'A', df['LTeamID'],
                                df['WTeamID']))  # For neutral, just pick one
    
    df['AwayTeamID'] = np.where(df['WLoc'] == 'H', df['LTeamID'],
                        np.where(df['WLoc'] == 'A', df['WTeamID'],
                                df['LTeamID']))
    
    df['HomeScore'] = np.where(df['WLoc'] == 'H', df['WScore'],
                      np.where(df['WLoc'] == 'A', df['LScore'],
                              df['WScore']))  # For neutral, just pick one
    
    df['AwayScore'] = np.where(df['WLoc'] == 'H', df['LScore'],
                      np.where(df['WLoc'] == 'A', df['WScore'],
                              df['LScore']))
    
    # Point spread from home team perspective
    # Positive = home team won by X points
    # Negative = home team lost by X points
    df['HomeSpread'] = df['HomeScore'] - df['AwayScore']
    
    # Mark neutral site games
    df['IsNeutral'] = (df['WLoc'] == 'N').astype(int)
    
    return df

df_games = calculate_point_spread(df_teams)

In [ ]:
def get_ranking_for_game(df_games, df_massey, ranking_systems=['POM']):
    """
    For each game on DayNum X, get rankings from the most recent RankingDayNum < X
    
    This ensures we only use information available BEFORE the game
    """
    
    # Filter to desired ranking systems
    rankings = df_massey[df_massey['SystemName'].isin(ranking_systems)].copy()
    
    # Sort by season, team, system, and ranking day
    rankings = rankings.sort_values(['Season', 'TeamID', 'SystemName', 'RankingDayNum'])
    
    # Create a lookup structure for faster access
    print("Creating ranking lookup structure...")
    rankings_dict = {}
    for system in ranking_systems:
        system_ranks = rankings[rankings['SystemName'] == system]
        for season in system_ranks['Season'].unique():
            season_ranks = system_ranks[system_ranks['Season'] == season]
            if season not in rankings_dict:
                rankings_dict[season] = {}
            rankings_dict[season][system] = season_ranks
    
    game_features = []
    skipped_games = 0

    total_games = len(df_games)
    
    for idx, (game_idx, game) in enumerate(df_games.iterrows()):
        
        season = game['Season']
        game_day = game['DayNum']
        home_team = game['HomeTeamID']
        away_team = game['AwayTeamID']
        
        # Check if we have rankings for this season
        if season not in rankings_dict:
            skipped_games += 1
            continue
        
        # Initialize features
        features = {
            'Season': season,
            'DayNum': game_day,
            'HomeTeamID': home_team,
            'AwayTeamID': away_team,
            'HomeSpread': game['HomeSpread'],
            'IsNeutral': game['IsNeutral']
        }
        
        has_all_rankings = True
        
        for system in ranking_systems:
            # Check if system exists for this season
            if system not in rankings_dict[season]:
                has_all_rankings = False
                break
            
            system_rankings = rankings_dict[season][system]
            
            # Get rankings BEFORE game day
            available_rankings = system_rankings[system_rankings['RankingDayNum'] < game_day]
            
            if len(available_rankings) == 0:
                has_all_rankings = False
                break
            
            # Home team ranking
            home_rankings = available_rankings[available_rankings['TeamID'] == home_team]
            if len(home_rankings) > 0:
                # Get most recent ranking
                latest_home = home_rankings.loc[home_rankings['RankingDayNum'].idxmax()]
                features[f'Home_{system}_Rank'] = latest_home['OrdinalRank']
                features[f'Home_{system}_RankDay'] = latest_home['RankingDayNum']
            else:
                has_all_rankings = False
                break
            
            # Away team ranking
            away_rankings = available_rankings[available_rankings['TeamID'] == away_team]
            if len(away_rankings) > 0:
                latest_away = away_rankings.loc[away_rankings['RankingDayNum'].idxmax()]
                features[f'Away_{system}_Rank'] = latest_away['OrdinalRank']
                features[f'Away_{system}_RankDay'] = latest_away['RankingDayNum']
            else:
                has_all_rankings = False
                break
            
            # Derived feature: ranking difference
            features[f'{system}_RankDiff'] = features[f'Away_{system}_Rank'] - features[f'Home_{system}_Rank']
        
        # Only add if we have all rankings
        if has_all_rankings:
            game_features.append(features)
        else:
            skipped_games += 1
    
    return pd.DataFrame(game_features)


# Use multiple ranking systems
ranking_systems = ['POM']

# Get features for all games
df_model = get_ranking_for_game(df_games, df_massey, ranking_systems)

In [ ]:
target_date_range=(90, 120)
training_seasons=range(2010, 2025)
training_data = df_model[
    (df_model['Season'].isin(training_seasons)) &
    (df_model['DayNum'] <= target_date_range[1])  # Use all games up to March
].copy()

test_data = df_model[
    (df_model['Season'] == 2025) &
    (df_model['DayNum'] >= target_date_range[0]) &
    (df_model['DayNum'] <= target_date_range[1])
].copy()

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def build_baseline_model(training_data, ranking_systems=['POM']):
    """
    Build baseline model using ranking features
    """
    
    # Define features
    feature_cols = []
    
    for system in ranking_systems:
        feature_cols.extend([
            f'Home_{system}_Rank',
            f'Away_{system}_Rank',
            f'{system}_RankDiff'
        ])
    
    # Add neutral site indicator
    feature_cols.append('IsNeutral')
    
    # Prepare X and y
    X = training_data[feature_cols].copy()
    y = training_data['HomeSpread'].copy()
    
    # Drop rows with missing values
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[mask]
    y = y[mask]
    
    print(f"\nTraining samples after removing NaNs: {len(X)}")
    print(f"Features: {feature_cols}")
    print(f"\nFeature statistics:")
    print(X.describe())
    
    return X, y, feature_cols


def train_and_evaluate_models(X_train, y_train, X_test, y_test):
    """
    Train multiple models and compare performance
    """
    
    models = {
        'Ridge': Ridge(alpha=1.0),
        'Lasso': Lasso(alpha=0.1),
        'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
    }
    
    results = []
    trained_models = {}
    
    for name, model in models.items():
        print(f"\n{'='*60}")
        print(f"Training {name}...")
        print('='*60)
        
        # Train
        model.fit(X_train, y_train)
        trained_models[name] = model
        
        # Predict on train
        train_pred = model.predict(X_train)
        train_mae = mean_absolute_error(y_train, train_pred)
        train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
        train_r2 = r2_score(y_train, train_pred)
        
        # Predict on test
        test_pred = model.predict(X_test)
        test_mae = mean_absolute_error(y_test, test_pred)
        test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
        test_r2 = r2_score(y_test, test_pred)
        
        # Cross-validation on training set
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, 
                                     scoring='neg_mean_absolute_error')
        cv_mae = -cv_scores.mean()
        
        results.append({
            'Model': name,
            'Train_MAE': train_mae,
            'Train_RMSE': train_rmse,
            'Train_R2': train_r2,
            'Test_MAE': test_mae,
            'Test_RMSE': test_rmse,
            'Test_R2': test_r2,
            'CV_MAE': cv_mae
        })
        
        print(f"Train MAE: {train_mae:.3f}, RMSE: {train_rmse:.3f}, R²: {train_r2:.3f}")
        print(f"Test MAE: {test_mae:.3f}, RMSE: {test_rmse:.3f}, R²: {test_r2:.3f}")
        print(f"CV MAE: {cv_mae:.3f}")
    
    results_df = pd.DataFrame(results)
    
    print("\n" + "="*80)
    print("MODEL COMPARISON")
    print("="*80)
    print(results_df.to_string(index=False))
    
    # Find best model
    best_model_name = results_df.loc[results_df['Test_MAE'].idxmin(), 'Model']
    print(f"\nBest model by Test MAE: {best_model_name}")
    
    return trained_models, results_df

X_train, y_train, feature_cols = build_baseline_model(training_data)

from sklearn.model_selection import train_test_split
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

trained_models, results_df = train_and_evaluate_models(
    X_train_split, y_train_split, X_val_split, y_val_split
)

In [ ]:
def analyze_feature_importance(trained_models, feature_cols):
    """
    Analyze which features are most important
    """
    
    # Get feature importance from tree-based models
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    for idx, (model_name, ax) in enumerate(zip(['Random Forest', 'Gradient Boosting'], axes)):
        if model_name in trained_models:
            model = trained_models[model_name]
            
            importances = model.feature_importances_
            indices = np.argsort(importances)[::-1][:15]  # Top 15 features
            
            ax.barh(range(len(indices)), importances[indices], color='steelblue', alpha=0.8)
            ax.set_yticks(range(len(indices)))
            ax.set_yticklabels([feature_cols[i] for i in indices])
            ax.set_xlabel('Feature Importance', fontweight='bold')
            ax.set_title(f'{model_name}\nTop 15 Features', fontweight='bold', fontsize=12)
            ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def plot_predictions(trained_models, X_val, y_val, model_name='Gradient Boosting'):
    """
    Plot predicted vs actual spreads
    """
    
    if model_name not in trained_models:
        print(f"Model {model_name} not found!")
        return
    
    model = trained_models[model_name]
    predictions = model.predict(X_val)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Scatter plot
    ax1 = axes[0]
    ax1.scatter(y_val, predictions, alpha=0.5, s=20)
    
    # Perfect prediction line
    min_val = min(y_val.min(), predictions.min())
    max_val = max(y_val.max(), predictions.max())
    ax1.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    
    ax1.set_xlabel('Actual Home Spread', fontweight='bold', fontsize=11)
    ax1.set_ylabel('Predicted Home Spread', fontweight='bold', fontsize=11)
    ax1.set_title(f'{model_name}\nPredicted vs Actual', fontweight='bold', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Residuals
    ax2 = axes[1]
    residuals = y_val - predictions
    ax2.scatter(predictions, residuals, alpha=0.5, s=20)
    ax2.axhline(y=0, color='r', linestyle='--', lw=2)
    ax2.set_xlabel('Predicted Home Spread', fontweight='bold', fontsize=11)
    ax2.set_ylabel('Residual (Actual - Predicted)', fontweight='bold', fontsize=11)
    ax2.set_title('Residual Plot', fontweight='bold', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print error distribution
    print("\n" + "="*60)
    print("PREDICTION ERROR ANALYSIS")
    print("="*60)
    print(f"Mean Absolute Error: {np.abs(residuals).mean():.3f}")
    print(f"Median Absolute Error: {np.median(np.abs(residuals)):.3f}")
    print(f"RMSE: {np.sqrt((residuals**2).mean()):.3f}")
    print(f"\nError percentiles:")
    print(f"  25th: {np.percentile(np.abs(residuals), 25):.3f}")
    print(f"  50th: {np.percentile(np.abs(residuals), 50):.3f}")
    print(f"  75th: {np.percentile(np.abs(residuals), 75):.3f}")
    print(f"  90th: {np.percentile(np.abs(residuals), 90):.3f}")


# Run analyses
analyze_feature_importance(trained_models, feature_cols)
plot_predictions(trained_models, X_val_split, y_val_split, 'Gradient Boosting')

In [ ]:
import xgboost as xgb

def create_comprehensive_features(df_model, ranking_systems=['POM', 'MOR', 'DOK', 'MAS', 'SAG']):
    """
    Create multiple representations of rankings with proper handling of edge cases
    """
    df = df_model.copy()
    
    max_rank = 351
    
    for system in ranking_systems:
        # 1. Keep original ranks (for tree models)
        # Already have: Home_{system}_Rank, Away_{system}_Rank, {system}_RankDiff
        
        # 2. Inverse strength (linear relationship)
        # Clip ranks to reasonable range to avoid negatives
        home_rank_clipped = np.clip(df[f'Home_{system}_Rank'], 1, max_rank)
        away_rank_clipped = np.clip(df[f'Away_{system}_Rank'], 1, max_rank)
        
        df[f'Home_{system}_Strength'] = max_rank - home_rank_clipped + 1
        df[f'Away_{system}_Strength'] = max_rank - away_rank_clipped + 1
        df[f'{system}_StrengthDiff'] = df[f'Home_{system}_Strength'] - df[f'Away_{system}_Strength']
        
        # 3. Log strength (diminishing returns at top)
        # Add small epsilon to avoid log(0), use clipped strengths
        df[f'Home_{system}_LogStrength'] = np.log(df[f'Home_{system}_Strength'] + 1)
        df[f'Away_{system}_LogStrength'] = np.log(df[f'Away_{system}_Strength'] + 1)
        
        # 4. Squared inverse (emphasize top teams)
        df[f'Home_{system}_Strength2'] = df[f'Home_{system}_Strength'] ** 2
        df[f'Away_{system}_Strength2'] = df[f'Away_{system}_Strength'] ** 2
        
        # 5. Boolean indicators for elite teams
        df[f'Home_{system}_IsTop25'] = (df[f'Home_{system}_Rank'] <= 25).astype(int)
        df[f'Away_{system}_IsTop25'] = (df[f'Away_{system}_Rank'] <= 25).astype(int)
        df[f'Home_{system}_IsTop50'] = (df[f'Home_{system}_Rank'] <= 50).astype(int)
        df[f'Away_{system}_IsTop50'] = (df[f'Away_{system}_Rank'] <= 50).astype(int)
        
        # 6. Matchup indicators
        df[f'{system}_BothTop25'] = (df[f'Home_{system}_IsTop25'] & df[f'Away_{system}_IsTop25']).astype(int)
        df[f'{system}_BothTop50'] = (df[f'Home_{system}_IsTop50'] & df[f'Away_{system}_IsTop50']).astype(int)
    
    # 7. Consensus features across ranking systems
    rank_cols = [f'Home_{s}_Rank' for s in ranking_systems]
    df['Home_AvgRank'] = df[rank_cols].mean(axis=1)
    df['Home_StdRank'] = df[rank_cols].std(axis=1)  # Disagreement among rankers
    
    rank_cols = [f'Away_{s}_Rank' for s in ranking_systems]
    df['Away_AvgRank'] = df[rank_cols].mean(axis=1)
    df['Away_StdRank'] = df[rank_cols].std(axis=1)
    
    df['AvgRankDiff'] = df['Away_AvgRank'] - df['Home_AvgRank']
    
    # 8. Strength ratio features
    df['Home_BestRank'] = df[[f'Home_{s}_Rank' for s in ranking_systems]].min(axis=1)
    df['Away_BestRank'] = df[[f'Away_{s}_Rank' for s in ranking_systems]].min(axis=1)
    df['Home_WorstRank'] = df[[f'Home_{s}_Rank' for s in ranking_systems]].max(axis=1)
    df['Away_WorstRank'] = df[[f'Away_{s}_Rank' for s in ranking_systems]].max(axis=1)
    
    return df



ranking_systems = ['POM']
df_model_full = create_comprehensive_features(df_model, ranking_systems)

# Step 2: Define feature sets
def get_feature_columns(ranking_systems):
    """
    Define all features to use in the model
    """
    feature_cols = ['IsNeutral']
    
    for system in ranking_systems:
        feature_cols.extend([
            # Original ranks
            f'Home_{system}_Rank',
            f'Away_{system}_Rank',
            f'{system}_RankDiff',
            
            # Strength transformations
            f'Home_{system}_Strength',
            f'Away_{system}_Strength',
            f'{system}_StrengthDiff',
            
            # Log transformations
            f'Home_{system}_LogStrength',
            f'Away_{system}_LogStrength',
            
            # Squared strength
            f'Home_{system}_Strength2',
            f'Away_{system}_Strength2',
            
            # Top team indicators
            f'Home_{system}_IsTop25',
            f'Away_{system}_IsTop25',
            f'Home_{system}_IsTop50',
            f'Away_{system}_IsTop50',
            
            # Matchup indicators
            f'{system}_BothTop25',
            f'{system}_BothTop50'
        ])
    
    # Add consensus features
    feature_cols.extend([
        'Home_AvgRank',
        'Away_AvgRank',
        'Home_StdRank',
        'Away_StdRank',
        'AvgRankDiff',
        'Home_BestRank',
        'Away_BestRank',
        'Home_WorstRank',
        'Away_WorstRank'
    ])
    
    return feature_cols

feature_cols = get_feature_columns(ranking_systems)


# Step 3: Prepare train/val/test splits
def prepare_data_splits(df_model_full, feature_cols, 
                       train_seasons=range(2010, 2022),
                       val_seasons=[2022, 2023],
                       test_season=2024,
                       target_day_range=(90, 120)):
    """
    Split data into train, validation, and test sets
    """
    
    # Training data: historical seasons, all days up to day 120
    train_data = df_model_full[
        (df_model_full['Season'].isin(train_seasons)) &
        (df_model_full['DayNum'] <= 120)
    ].copy()
    
    # Validation data: recent seasons in target day range
    val_data = df_model_full[
        (df_model_full['Season'].isin(val_seasons)) &
        (df_model_full['DayNum'] >= target_day_range[0]) &
        (df_model_full['DayNum'] <= target_day_range[1])
    ].copy()
    
    # Test data: most recent season in target day range
    test_data = df_model_full[
        (df_model_full['Season'] == test_season) &
        (df_model_full['DayNum'] >= target_day_range[0]) &
        (df_model_full['DayNum'] <= target_day_range[1])
    ].copy()
    
    # Extract X and y
    X_train = train_data[feature_cols].copy()
    y_train = train_data['HomeSpread'].copy()
    
    X_val = val_data[feature_cols].copy()
    y_val = val_data['HomeSpread'].copy()
    
    X_test = test_data[feature_cols].copy()
    y_test = test_data['HomeSpread'].copy()
    
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = prepare_data_splits(
    df_model_full, 
    feature_cols,
    train_seasons=range(2015, 2023),
    val_seasons=[2023, 2024],
    test_season=2025,
    target_day_range=(90, 120)
)

# Step 4: Define monotonic constraints
def get_monotonic_constraints(feature_cols, ranking_systems):
    """
    Define monotonic constraints for ranking features
    
    Logic:
    - Higher HOME rank (worse) → LOWER home spread → constraint = -1
    - Higher AWAY rank (worse) → HIGHER home spread → constraint = +1
    - Positive RankDiff (away worse) → HIGHER home spread → constraint = +1
    - Strength is inverse of rank, so opposite constraints
    """
    
    constraints = []
    
    for col in feature_cols:
        if col == 'IsNeutral':
            constraints.append(0)  # No constraint
        
        # Original rank features
        elif '_Rank' in col and 'Home_' in col and 'Avg' not in col and 'Best' not in col and 'Worst' not in col:
            constraints.append(-1)  # Higher home rank → lower spread
        elif '_Rank' in col and 'Away_' in col and 'Avg' not in col and 'Best' not in col and 'Worst' not in col:
            constraints.append(1)   # Higher away rank → higher spread
        elif 'RankDiff' in col:
            constraints.append(1)   # Positive diff (away worse) → higher spread
        
        # Strength features (inverse of rank)
        elif 'Strength' in col and 'Home_' in col and 'Diff' not in col:
            constraints.append(1)   # Higher home strength → higher spread
        elif 'Strength' in col and 'Away_' in col and 'Diff' not in col:
            constraints.append(-1)  # Higher away strength → lower spread
        elif 'StrengthDiff' in col:
            constraints.append(1)   # Positive diff (home stronger) → higher spread
        
        # Average/consensus ranks
        elif 'Home_AvgRank' in col:
            constraints.append(-1)
        elif 'Away_AvgRank' in col:
            constraints.append(1)
        elif 'AvgRankDiff' in col:
            constraints.append(1)
        
        # Best/Worst ranks
        elif 'Home_BestRank' in col or 'Home_WorstRank' in col:
            constraints.append(-1)
        elif 'Away_BestRank' in col or 'Away_WorstRank' in col:
            constraints.append(1)
        
        # No constraint for other features (top25 indicators, std, etc.)
        else:
            constraints.append(0)
    
    return tuple(constraints)

monotonic_constraints = get_monotonic_constraints(feature_cols, ranking_systems)


# Create DMatrix objects
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_cols)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=feature_cols)
dtest = xgb.DMatrix(X_test, label=y_test, feature_names=feature_cols)

# XGBoost parameters
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'monotone_constraints': monotonic_constraints,
    'max_depth': 6,
    'eta': 0.05,  # Learning rate
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'gamma': 0.1,
    'lambda': 1.0,  # L2 regularization
    'alpha': 0.1,   # L1 regularization
    'seed': 42
}

for key, value in params.items():
    if key != 'monotone_constraints':
        print(f"  {key}: {value}")

# Train model
evals = [(dtrain, 'train'), (dval, 'val')]

xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=evals,
    early_stopping_rounds=50,
    verbose_eval=50
)


# Step 6: Evaluate model
def evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, feature_cols):
    """
    Comprehensive model evaluation
    """
    
    # Create DMatrix for predictions
    dtrain = xgb.DMatrix(X_train, feature_names=feature_cols)
    dval = xgb.DMatrix(X_val, feature_names=feature_cols)
    dtest = xgb.DMatrix(X_test, feature_names=feature_cols)
    
    # Predictions
    train_pred = model.predict(dtrain)
    val_pred = model.predict(dval)
    test_pred = model.predict(dtest)
    
    # Calculate metrics
    results = {
        'Train': {
            'MAE': mean_absolute_error(y_train, train_pred),
            'RMSE': np.sqrt(mean_squared_error(y_train, train_pred)),
            'R2': r2_score(y_train, train_pred),
            'N': len(y_train)
        },
        'Validation': {
            'MAE': mean_absolute_error(y_val, val_pred),
            'RMSE': np.sqrt(mean_squared_error(y_val, val_pred)),
            'R2': r2_score(y_val, val_pred),
            'N': len(y_val)
        },
        'Test': {
            'MAE': mean_absolute_error(y_test, test_pred),
            'RMSE': np.sqrt(mean_squared_error(y_test, test_pred)),
            'R2': r2_score(y_test, test_pred),
            'N': len(y_test)
        }
    }
    
    
    results_df = pd.DataFrame(results).T
    print(results_df.to_string())
    
    # Residual analysis

    
    test_residuals = y_test - test_pred
    
    print(f"Mean Error: {test_residuals.mean():.3f}")
    print(f"Median Absolute Error: {np.median(np.abs(test_residuals)):.3f}")
    print(f"Std of Errors: {test_residuals.std():.3f}")
    print(f"\nError Percentiles:")
    print(f"  10th: {np.percentile(np.abs(test_residuals), 10):.3f}")
    print(f"  25th: {np.percentile(np.abs(test_residuals), 25):.3f}")
    print(f"  50th: {np.percentile(np.abs(test_residuals), 50):.3f}")
    print(f"  75th: {np.percentile(np.abs(test_residuals), 75):.3f}")
    print(f"  90th: {np.percentile(np.abs(test_residuals), 90):.3f}")
    
    # Within X points accuracy
    for threshold in [3, 5, 7, 10]:
        within = (np.abs(test_residuals) <= threshold).mean() * 100
        print(f"  {threshold} points: {within:.1f}%")
    
    return results_df, train_pred, val_pred, test_pred

results_df, train_pred, val_pred, test_pred = evaluate_model(
    xgb_model, X_train, y_train, X_val, y_val, X_test, y_test, feature_cols
)

# Step 7: Feature importance analysis
def plot_feature_importance(model, feature_cols, top_n=20):
    """
    Plot feature importance
    """
    
    importance = model.get_score(importance_type='gain')
    
    # Convert to dataframe
    importance_df = pd.DataFrame([
        {'feature': k, 'importance': v} 
        for k, v in importance.items()
    ]).sort_values('importance', ascending=False)
    
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 8))
    
    top_features = importance_df.head(top_n)
    ax.barh(range(len(top_features)), top_features['importance'], color='steelblue', alpha=0.8)
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['feature'])
    ax.set_xlabel('Importance (Gain)', fontweight='bold', fontsize=12)
    ax.set_title(f'Top {top_n} Most Important Features', fontweight='bold', fontsize=14)
    ax.grid(axis='x', alpha=0.3)
    ax.invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
    return importance_df

importance_df = plot_feature_importance(xgb_model, feature_cols, top_n=20)

# Step 8: Prediction visualizations
def plot_predictions(y_true, y_pred, set_name='Test'):
    """
    Visualize predictions vs actual
    """
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # 1. Scatter plot
    ax1 = axes[0, 0]
    ax1.scatter(y_true, y_pred, alpha=0.5, s=20)
    
    # Perfect prediction line
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    ax1.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    
    ax1.set_xlabel('Actual Home Spread', fontweight='bold', fontsize=11)
    ax1.set_ylabel('Predicted Home Spread', fontweight='bold', fontsize=11)
    ax1.set_title(f'{set_name} Set: Predicted vs Actual\nMAE: {mean_absolute_error(y_true, y_pred):.2f}', 
                 fontweight='bold', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Residual plot
    ax2 = axes[0, 1]
    residuals = y_true - y_pred
    ax2.scatter(y_pred, residuals, alpha=0.5, s=20)
    ax2.axhline(y=0, color='r', linestyle='--', lw=2)
    ax2.set_xlabel('Predicted Home Spread', fontweight='bold', fontsize=11)
    ax2.set_ylabel('Residual (Actual - Predicted)', fontweight='bold', fontsize=11)
    ax2.set_title('Residual Plot', fontweight='bold', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # 3. Residual distribution
    ax3 = axes[1, 0]
    ax3.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
    ax3.axvline(x=0, color='r', linestyle='--', lw=2)
    ax3.set_xlabel('Residual', fontweight='bold', fontsize=11)
    ax3.set_ylabel('Frequency', fontweight='bold', fontsize=11)
    ax3.set_title(f'Residual Distribution\nMean: {residuals.mean():.2f}, Std: {residuals.std():.2f}', 
                 fontweight='bold', fontsize=12)
    ax3.grid(axis='y', alpha=0.3)
    
    # 4. Error by prediction magnitude
    ax4 = axes[1, 1]
    
    # Bin predictions
    pred_bins = pd.cut(y_pred, bins=10)
    error_by_bin = pd.DataFrame({
        'pred': y_pred,
        'error': np.abs(residuals),
        'bin': pred_bins
    }).groupby('bin')['error'].mean()
    
    bin_centers = [interval.mid for interval in error_by_bin.index]
    ax4.plot(bin_centers, error_by_bin.values, marker='o', linewidth=2, markersize=8)
    ax4.set_xlabel('Predicted Spread (binned)', fontweight='bold', fontsize=11)
    ax4.set_ylabel('Mean Absolute Error', fontweight='bold', fontsize=11)
    ax4.set_title('Error by Prediction Magnitude', fontweight='bold', fontsize=12)
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_predictions(y_test, test_pred, 'Test')
